# cross_val_score()

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
import numpy as np

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state = 2023)

data = iris_data.data
label = iris_data.target

# 성능 지표는 정확도, cv는 3개
scores = cross_val_score(dt_clf, data, label, scoring = 'accuracy', cv = 3)

print(type(scores)) # numpy 타입이라 np.round로 반올림 해야한다
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

<class 'numpy.ndarray'>
교차 검증별 정확도: [0.98 0.92 0.98]
평균 검증 정확도: 0.96


# GridSearchCV

In [12]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# 데이터 불러오고 분리하기
iris = load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris_data.data,
                                                    iris_data.target,
                                                    test_size = 0.2,
                                                    random_state = 2023)

dtree = DecisionTreeClassifier()

# 파라미터를 딕셔너리 형태로 설정(하나여도 리스트로 해야한다)
parameters = {'max_depth' : [1, 2, 3], 'min_samples_split' : [2, 3]}

In [15]:
import pandas as pd

# 하이퍼 파라미터들을 3개의 train, test set fold 로 나눠서 테스트 수행 설정
# refit = True가 디폴트 값 : 가장 좋은 파라미터 설정으로 재 학습
grid_dtree = GridSearchCV(dtree, param_grid = parameters, cv = 3, refit = True, return_train_score = True)

grid_dtree.fit(X_train, y_train)

scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score']]

,params,mean_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.675000,5,0.6750,0.675,0.6750
1,"{'max_depth': 1, 'min_samples_split': 3}",0.675000,5,0.6750,0.675,0.6750
2,"{'max_depth': 2, 'min_samples_split': 2}",0.933333,1,0.9625,0.950,0.9625
3,"{'max_depth': 2, 'min_samples_split': 3}",0.933333,1,0.9625,0.950,0.9625
4,"{'max_depth': 3, 'min_samples_split': 2}",0.933333,1,0.9750,0.975,0.9750
5,"{'max_depth': 3, 'min_samples_split': 3}",0.933333,1,0.9750,0.975,0.9750


In [19]:
print('최적의 파라미터:', grid_dtree.best_params_)
print('최고 정확도:, {0: .4f}'.format(grid_dtree.best_score_))

pred = grid_dtree.predict(X_test)
print('테스트 데이터 세트 정확도: {0: .4f}'.format(accuracy_score(y_test, pred)))

최적의 파라미터: {'max_depth': 2, 'min_samples_split': 2}
최고 정확도:,  0.9333
테스트 데이터 세트 정확도:  1.0000


In [21]:
# 위와 같음
estimator = grid_dtree.best_estimator_
pred = estimator.predict(X_test)
accuracy_score(y_test, pred)

1.0

# 데이터 인코딩

## 레이블 인코딩

In [23]:
from sklearn.preprocessing import LabelEncoder

items=['TV','냉장고','전자렌지','컴퓨터','선풍기','선풍기','믹서','믹서']

# LabelEncoder를 객체로 생성한 후 , fit( ) 과 transform( ) 으로 label 인코딩 수행. 
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
print('인코딩 변환값:',labels)

인코딩 변환값: [0 1 4 5 3 3 2 2]


In [29]:
# fit, transform 도 한번에 가능
labels = encoder.fit_transform(items)
print(labels)

[0 1 4 5 3 3 2 2]


c:\Users\82105\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [26]:
print('인코딩 클래스:', encoder.classes_)
print('디코딩 원본 값:',encoder.inverse_transform([0, 1, 4, 5, 3, 3, 2, 2]))

인코딩 클래스: ['TV' '냉장고' '믹서' '선풍기' '전자렌지' '컴퓨터']
디코딩 원본 값: ['TV' '냉장고' '전자렌지' '컴퓨터' '선풍기' '선풍기' '믹서' '믹서']


## 원-핫 인코딩

In [28]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

items=['TV','냉장고','전자렌지','컴퓨터','선풍기','선풍기','믹서','믹서']

# 2차원 ndarray로 변환 
items = np.array(items).reshape(-1, 1)

# 원-핫 인코딩을 적용
oh_encoder = OneHotEncoder()
oh_encoder.fit(items)
oh_labels = oh_encoder.transform(items)

# OneHotEncoder로 변환한 결과는 희소행렬이므로 toarray()를 이용하여 밀집 행렬로 변환. 
print('원-핫 인코딩 데이터')
print(oh_labels.toarray())
print('원-핫 인코딩 데이터 차원')
print(oh_labels.shape)

원-핫 인코딩 데이터
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
원-핫 인코딩 데이터 차원
(8, 6)


In [30]:
# get_dummies() 로 쉽게 가능하다
import pandas as pd

df = pd.DataFrame({'item':['TV','냉장고','전자렌지','컴퓨터','선풍기','선풍기','믹서','믹서'] })
pd.get_dummies(df)

,item_TV,item_냉장고,item_믹서,item_선풍기,item_전자렌지,item_컴퓨터
0,True,False,False,False,False,False
1,False,True,False,False,False,False
2,False,False,False,False,True,False
3,False,False,False,False,False,True
4,False,False,False,True,False,False
5,False,False,False,True,False,False
6,False,False,True,False,False,False
7,False,False,True,False,False,False


# 피처 스케일링과 정규화

## StandardScaler

In [1]:
from sklearn.datasets import load_iris
import pandas as pd
# 붓꽃 데이터 셋을 로딩하고 DataFrame으로 변환
iris = load_iris()
iris_data = iris.data
iris_df = pd.DataFrame(data=iris_data, columns=iris.feature_names)

print('feature 들의 평균 :')
print(iris_df.mean())
print('\nfeature 들의 분산 :')
print(iris_df.var())

feature 들의 평균 :
sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
dtype: float64

feature 들의 분산 :
sepal length (cm)    0.685694
sepal width (cm)     0.189979
petal length (cm)    3.116278
petal width (cm)     0.581006
dtype: float64


In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(iris_df) # df도 가능하다.
iris_scaled = scaler.transform(iris_df)

iris_df_scaled = pd.DataFrame(data = iris_scaled, columns = iris.feature_names)

# 0만큼 작게 나온다
print('feature 들의 평균 :')
print(iris_df_scaled.mean())
print('\nfeature 들의 분산 :')
print(iris_df_scaled.var())

feature 들의 평균 :
sepal length (cm)   -1.690315e-15
sepal width (cm)    -1.842970e-15
petal length (cm)   -1.698641e-15
petal width (cm)    -1.409243e-15
dtype: float64

feature 들의 분산 :
sepal length (cm)    1.006711
sepal width (cm)     1.006711
petal length (cm)    1.006711
petal width (cm)     1.006711
dtype: float64


## MinMaxScaler

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

# transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)
print('feature들의 최소 값')
print(iris_df_scaled.min())
print('\nfeature들의 최대 값')
print(iris_df_scaled.max())

feature들의 최소 값
sepal length (cm)    0.0
sepal width (cm)     0.0
petal length (cm)    0.0
petal width (cm)     0.0
dtype: float64

feature들의 최대 값
sepal length (cm)    1.0
sepal width (cm)     1.0
petal length (cm)    1.0
petal width (cm)     1.0
dtype: float64
